## Check that GPU is available to TensorFlow

In [3]:
import tensorflow as tf
len(tf.config.list_physical_devices('GPU'))
# assert tf.test.is_gpu_available()
# assert tf.test.is_built_with_cuda()

1

In [2]:
# CNN
import pandas as pd
import numpy as np
import deepchem as dc
from tqdm import tqdm, trange
import tensorflow as tf
#from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import sys, os
from pathlib import Path
from deepchem.models.optimizers import Adam, ExponentialDecay

# Load dataset
pathway = Path()
sigma_csv = "../datasets/VT_2005_sigma_smiles_cid.csv"
sigma_csv = pd.read_csv(sigma_csv)
sigma_df = pd.DataFrame(sigma_csv)
smile = sigma_df["SMILE"]
smile_list = smile.tolist()
vocab_smile = set()
index = sigma_df["Index"]
txt_file_path = "../datasets/VT-2005_Sigma_Profiles_v2"
dict_sigma = {}
counter = 0

tasks, datasets, transformers = dc.molnet.load_chembl25(splitter='stratified')
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
test_smiles = test_dataset.ids

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(c for c in s))
    
    
tokens = sorted(list(tokens))


max_length = max(len(s) for s in train_smiles)
batch_size = 1024
embed=256
batches_per_epoch = len(train_smiles)/batch_size
print("done")


model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=embed,
                           model_dir='../chembl25/chembl25-weights_100epochs',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))


model.restore()

# create vocabulary of smiles
# for x in smile_list:
#     # print(x)
#     for y in x:
#         try:
#             vocab_smile.add(y)
#         except:
#             continue

# print(vocab_smile)
# vocab_smile = list(vocab_smile)
# input_tokens = sorted(vocab_smile)

# create a dictionary of indices and corresponding sigma profiles
for i in index:
    i_str = str(i)
    i_str = i_str.zfill(4)
    for file in pathway.glob(f"{txt_file_path}/VT2005-{i_str}-PROF.txt"):
        # print(file)
        sigma_file = pd.read_csv(file, sep='\s+', header=None)
        sigma_df = pd.DataFrame(sigma_file)
        sigma_1 = sigma_df[1]
        sigma1_np = sigma_1.to_numpy()
        dict_sigma[i_str] = list(sigma1_np)
        # print(sigma1_np)
        # print(dict_sigma)
        counter +=1

print(counter)
# the dataset for VT-2005 index and corresponding sigma-profile
dataset_index = [key for key,val in dict_sigma.items()]
dataset_sigma = [val for key,val in dict_sigma.items()]
assert len(dataset_sigma) == len(dataset_index)  # verify that the lengths are equal


done
1204


In [4]:
# from deepchem.feat.smiles_tokenizer import BasicSmilesTokenizer
from deepchem.feat import BasicSmilesTokenizer

tokenizer = BasicSmilesTokenizer()
print(tokenizer.tokenize("CC(=O)OC1=CC=CC=C1C(=O)O"))
model.predict_embeddings(tokenizer.tokenize("CC(=O)OC1=CC=CC=C1C(=O)O")).shape

ImportError: cannot import name 'BasicSmilesTokenizer' from 'deepchem.feat' (/usr/local/Miniconda3/envs/deepchem-2.5.0-gpu/lib/python3.8/site-packages/deepchem/feat/__init__.py)

In [ ]:
# model training

predicted = model.predict_from_sequences(valid_smiles)
count = 0
for s,p in tqdm(zip(valid_smiles, predicted)):
    if ''.join(p) == s:
        count += 1
print('reproduced', count, f'of {len(valid_smiles)} validation SMILES strings')

### Test an encoder-decoder (trained on MUV dataset) on the VT-2005 SMILES dataset

In [1]:
"""
This script tests an encoder-decoder (trained on MUV) on a list of smiles from VT-2005 and tries to predict smiles using 
their embedding vectors.
"""
import pandas as pd
import numpy as np
import deepchem as dc
from tqdm import tqdm, trange
import tensorflow as tf
#from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import sys, os
from pathlib import Path
from deepchem.models.optimizers import Adam, ExponentialDecay

# Load dataset
pathway = Path()
sigma_csv = "../datasets/VT_2005_sigma_smiles_cid.csv"
sigma_csv = pd.read_csv(sigma_csv)
sigma_df = pd.DataFrame(sigma_csv)
smile = sigma_df["SMILE"]
smile_list = smile.tolist()
vocab_smile = set()
index = sigma_df["Index"]
txt_file_path = "./datasets/VT-2005_Sigma_Profiles_v2"
dict_sigma = {}
counter = 0

tasks, datasets, transformers = dc.molnet.load_muv(splitter='stratified')
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))


max_length = max(len(s) for s in train_smiles)
batch_size = 100
embed=256
batches_per_epoch = len(train_smiles)/batch_size

model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=embed,
                           model_dir='../muv-256_100epochs/muv-256-weights_100epochs',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))


model.restore()

# create vocabulary of smiles
for x in smile_list:
    # print(x)
    for y in x:
        try:
            vocab_smile.add(y)
        except:
            continue

print(vocab_smile)
vocab_smile = list(vocab_smile)
input_tokens = sorted(vocab_smile)

# create a dictionary of indices and corresponding sigma profiles
for i in index:
    i_str = str(i)
    i_str = i_str.zfill(4)
    for file in pathway.glob(f"{txt_file_path}/VT2005-{i_str}-PROF.txt"):
        # print(file)
        sigma_file = pd.read_csv(file, sep='\s+', header=None)
        sigma_df = pd.DataFrame(sigma_file)
        sigma_1 = sigma_df[1]
        sigma1_np = sigma_1.to_numpy()
        dict_sigma[i_str] = list(sigma1_np)
        # print(sigma1_np)
        # print(dict_sigma)
        counter +=1

print(counter)
# the dataset for VT-2005 index and corresponding sigma-profile
dataset_index = [key for key,val in dict_sigma.items()]
dataset_sigma = [val for key,val in dict_sigma.items()]
assert len(dataset_sigma) == len(dataset_index)  # verify that the lengths are equal


predictions = []
good_smiles = []
bad = False
for smile in smile_list:
    for i in smile:
        if i not in tokens:
            bad = True

    if not bad:
        #predictions.append(model.predict_from_sequences(smile))
        good_smiles.append(smile)
        # print(smile)
    # else:
    #     predictions.append(model.predict_from_sequences(smile_list))

predictions = model.predict_from_sequences(good_smiles)
count = 0
for s,p in tqdm(zip(good_smiles, predictions)):
    if ''.join(p) == s:
    count += 1
print('reproduced', count, f'of {len(good_smiles)} validation SMILES strings')

2022-04-01 13:18:51.233800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

2022-04-01 13:23:36.874770: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-01 13:23:36.894202: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-01 13:23:37.835359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-04-01 13:23:37.835445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-01 13:23:38.161176: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-04-01 13:23:38.161352: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2

{'2', '4', 'r', '1', '#', 'C', 'O', '=', '[', 'H', 'l', 'S', '-', '3', 'B', 'I', ')', ']', 'F', '5', 'P', '+', 'N', '('}
0


2022-04-01 13:23:41.545387: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-01 13:23:41.557922: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
714it [00:00, 1093766.64it/s]

reproduced 41 of 714 validation SMILES strings


### Test an encoder-decoder (trained on CHEMBL25 dataset) on the VT-2005 SMILES dataset

In [7]:
# VT-2005 
"""
This script tests an encoder-decoder (trained on chembl25) on a list of smiles from VT-2005 and tries to predict smiles using 
their embedding vectors.
"""
import pandas as pd
import numpy as np
import deepchem as dc
from tqdm import tqdm, trange
import tensorflow as tf
#from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import sys, os
from pathlib import Path
from deepchem.models.optimizers import Adam, ExponentialDecay

# Load dataset
pathway = Path()
sigma_csv = "../datasets/VT_2005_sigma_smiles_cid.csv"
sigma_csv = pd.read_csv(sigma_csv)
sigma_df = pd.DataFrame(sigma_csv)
smile = sigma_df["SMILE"]
smile_list = smile.tolist()
vocab_smile = set()
index = sigma_df["Index"]
txt_file_path = "../datasets/VT-2005_Sigma_Profiles_v2"
dict_sigma = {}
counter = 0

# tasks, datasets, transformers = dc.molnet.load_chembl25(splitter='stratified')
# train_dataset, valid_dataset, test_dataset = datasets
# train_smiles = train_dataset.ids
# valid_smiles = valid_dataset.ids
# test_smiles = test_dataset.ids

# Load CHEMBL25 from joblib files instead. Probably faster than molnet
train_dataset = dc.utils.load_from_disk('../datasets/chembl25/chembl25_strat_train.joblib')
train_smiles = train_dataset.ids
valid_dataset = dc.utils.load_from_disk('../datasets/chembl25/chembl25_strat_valid.joblib')
valid_smiles = valid_dataset.ids
test_dataset = dc.utils.load_from_disk('../datasets/chembl25/chembl25_strat_test.joblib')
test_smiles = test_dataset.ids
print("Train: ",len(train_smiles))
print("Valid: ",len(valid_smiles))
print("Test: ",len(test_smiles))

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))


max_length = max(len(s) for s in train_smiles)
batch_size = 1024
embed=256
batches_per_epoch = len(train_smiles)/batch_size
print("done")


model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=embed,
                           model_dir='../chembl25/chembl25-weights_100epochs',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))


model.restore()

# create vocabulary of smiles
for x in smile_list:
    # print(x)
    for y in x:
        try:
            vocab_smile.add(y)
        except:
            continue

print(vocab_smile)
vocab_smile = list(vocab_smile)
input_tokens = sorted(vocab_smile)

# create a dictionary of indices and corresponding sigma profiles
for i in index:
    i_str = str(i)
    i_str = i_str.zfill(4)
    for file in pathway.glob(f"{txt_file_path}/VT2005-{i_str}-PROF.txt"):
        # print(file)
        sigma_file = pd.read_csv(file, sep='\s+', header=None)
        sigma_df = pd.DataFrame(sigma_file)
        sigma_1 = sigma_df[1]
        sigma1_np = sigma_1.to_numpy()
        dict_sigma[i_str] = list(sigma1_np)
        # print(sigma1_np)
        # print(dict_sigma)
        counter +=1

print(counter)
# the dataset for VT-2005 index and corresponding sigma-profile
dataset_index = [key for key,val in dict_sigma.items()]
dataset_sigma = [val for key,val in dict_sigma.items()]
assert len(dataset_sigma) == len(dataset_index)  # verify that the lengths are equal


predictions = []
good_smiles = []
bad = False
for smile in smile_list:
    for i in smile:
        if i not in tokens:
            bad = True

    if not bad:
        #predictions.append(model.predict_from_sequences(smile))
        good_smiles.append(smile)
        # print(smile)
    # else:
    #     predictions.append(model.predict_from_sequences(smile_list))

predictions = model.predict_from_sequences(good_smiles)
count = 0
for s,p in tqdm(zip(good_smiles, predictions)):
    if ''.join(p) == s:
    count += 1
print('reproduced', count, f'of {len(good_smiles)} validation SMILES strings')

{'2', '4', 'r', '1', '#', 'C', 'O', '=', '[', 'H', 'l', 'S', '-', '3', 'B', 'I', ')', ']', 'F', '5', 'P', '+', 'N', '('}
0


1204it [00:00, 1053607.76it/s]

reproduced 90 of 1204 validation SMILES strings


### Load CHEMBL25 dataset and save as joblib

In [8]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_chembl25(splitter='stratified')
train, valid, test = datasets
dc.utils.save_to_disk(datasets, 'chembl25_strat.joblib')
dc.utils.save_to_disk(train, 'chembl25_strat_train.joblib')
dc.utils.save_to_disk(test, 'chembl25_strat_test.joblib')
dc.utils.save_to_disk(valid, 'chembl25_strat_valid.joblib')
train_dataset = dc.utils.load_from_disk('chembl25_strat_train.joblib')
train_smiles = train_dataset.ids
valid_dataset = dc.utils.load_from_disk('chembl25_strat_valid.joblib')
valid_smiles = valid_dataset.ids
test_dataset = dc.utils.load_from_disk('chembl25_strat_test.joblib')
test_smiles = test_dataset.ids
print("Train",len(train_smiles))
print("Valid",len(valid_smiles))
print("Test",len(test_smiles))

RDKit WARNING: [13:35:42] WARNING: not removing hydrogen atom without neighbors
[13:35:42] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [13:44:51] WARNING: not removing hydrogen atom without neighbors
[13:44:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [13:45:48] WARNING: not removing hydrogen atom without neighbors
[13:45:48] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [13:46:40] WARNING: not removing hydrogen atom without neighbors
[13:46:40] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:01:14] WARNING: not removing hydrogen atom without neighbors
[14:01:14] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:01:14] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:01:14] WARNING: not removing hydrogen atom without neighbors
[14:01:14] WARNING: not removing hydrogen atom without neighbors
[14:01:14] WARNING: not removing hydrogen atom wit

Train 1428330
Valid 178543
Test 178542


In [17]:
train_dataset.ids[0]

'Cc1cc(-c2csc(N=C(N)N)n2)cn1C'

### Test an encoder-decoder (trained on CHEMBL25 for 3 epochs) on the test/validation SMILES from CHEMBL25 dataset.
Training set: 1,428,327 
Test set: 178,541 
Valid set: 178, 547

In [18]:
# No VT-2005 
"""
This script tests an encoder-decoder (trained on chembl25) and tries to predict smiles using 
their embedding vectors.
"""
import pandas as pd
import numpy as np
import deepchem as dc
from tqdm import tqdm, trange
import tensorflow as tf
#from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import sys, os
from pathlib import Path
from deepchem.models.optimizers import Adam, ExponentialDecay

tasks, datasets, transformers = dc.molnet.load_chembl25(splitter='stratified')
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
test_smiles = test_dataset.ids

# Load CHEMBL25 from joblib files instead. Probably faster than molnet
# train_dataset = dc.utils.load_from_disk('./chembl25_strat_train.joblib')
# train_smiles = train_dataset.ids
# valid_dataset = dc.utils.load_from_disk('./chembl25_strat_valid.joblib')
# valid_smiles = valid_dataset.ids
# test_dataset = dc.utils.load_from_disk('./chembl25_strat_test.joblib')
# test_smiles = test_dataset.ids
print("Train: ",len(train_smiles))
print("Valid: ",len(valid_smiles))
print("Test: ",len(test_smiles))

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))


max_length = max(len(s) for s in train_smiles)
batch_size = 1024
embed=256
batches_per_epoch = len(train_smiles)/batch_size
print("done")


model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=embed,
                           model_dir='../chembl25/chembl25-weights_100epochs',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))


model.restore()

#validation set
predicted_valid = model.predict_from_sequences(valid_smiles)
count = 0
for s,p in tqdm(zip(valid_smiles, predicted_valid)):
    if ''.join(p) == s:
        count += 1
print('reproduced', count, f'of {len(valid_smiles)} validation SMILES strings')


# test set
predicted_test = model.predict_from_sequences(test_smiles)
count = 0
for s,p in tqdm(zip(test_smiles, predicted_test)):
    if ''.join(p) == s:
        count += 1
print('reproduced', count, f'of {len(test_smiles)} test SMILES strings')

Train:  1428327
Valid:  178547
Test:  178541
done


2022-04-13 12:43:19.865987: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-13 12:43:20.928561: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-13 12:43:21.780627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:18:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-04-13 12:43:21.780679: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-13 12:43:22.230475: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-04-13 12:43:22.230544: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2

reproduced 5994 of 178547 validation SMILES strings


178541it [00:00, 1354058.40it/s]

reproduced 5998 of 178541 test SMILES strings


### Test an encoder-decoder (trained on CHEMBL25 for 10 epochs) on the test/validation SMILES from CHEMBL25 dataset.
Training set: 1,428,327 
Test set: 178,541 
Valid set: 178, 547

In [ ]:
# No VT-2005 
"""
This script tests an encoder-decoder (trained for 10 epochs on chembl25) and tries to predict smiles using 
their embedding vectors.
"""
import pandas as pd
import numpy as np
import deepchem as dc
from tqdm import tqdm, trange
import tensorflow as tf
#from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import sys, os
from pathlib import Path
from deepchem.models.optimizers import Adam, ExponentialDecay

# Load CHEMBL25 from molnet
# tasks, datasets, transformers = dc.molnet.load_chembl25(splitter='stratified')
# train_dataset, valid_dataset, test_dataset = datasets


# Load CHEMBL25 from joblib files instead. Probably faster than molnet
train_dataset = dc.utils.load_from_disk('./chembl25_strat_train.joblib')
valid_dataset = dc.utils.load_from_disk('./chembl25_strat_valid.joblib')
test_dataset = dc.utils.load_from_disk('./chembl25_strat_test.joblib')

train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
test_smiles = test_dataset.ids
print("Train: ",len(train_smiles))
print("Valid: ",len(valid_smiles))
print("Test: ",len(test_smiles))

tokens = set()
for s in train_smiles:
    tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))


max_length = max(len(s) for s in train_smiles)
batch_size = 1024
embed=256
batches_per_epoch = len(train_smiles)/batch_size
print("done")


model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=embed,
                           model_dir='../chembl25/test-cuda/chembl25-weights_100epochs',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))


model.restore()

#validation set
predicted_valid = model.predict_from_sequences(valid_smiles)
count = 0
for s,p in tqdm(zip(valid_smiles, predicted_valid)):
    if ''.join(p) == s:
        count += 1
print('reproduced', count, f'of {len(valid_smiles)} validation SMILES strings')


# test set
predicted_test = model.predict_from_sequences(test_smiles)
count = 0
for s,p in tqdm(zip(test_smiles, predicted_test)):
    if ''.join(p) == s:
        count += 1
print('reproduced', count, f'of {len(test_smiles)} test SMILES strings')

Train:  1428330
Valid:  178543
Test:  178542
done


In [ ]:
#validation set
valid_smiles = valid_smiles
predicted_valid = model.predict_from_sequences(valid_smiles)
count = 0
for s,p in tqdm(zip(valid_smiles, predicted_valid)):
    if ''.join(p) == s:
    count += 1
print('reproduced', count, f'of {len(valid_smiles)} validation SMILES strings')


# test set
test_smiles = test_dataset.ids
predicted_test = model.predict_from_sequences(test_smiles)
count = 0
for s,p in tqdm(zip(test_smiles, predicted_test)):
    if ''.join(p) == s:
    count += 1
print('reproduced', count, f'of {len(test_smiles)} test SMILES strings')

10000it [00:00, 1006069.56it/s]


reproduced 421 of 10000 validation SMILES strings


In [3]:
embeddings = model.predict_embeddings(valid_smiles[0])


In [6]:
len(valid_smiles[0])

38

In [4]:
embeddings.shape

(38, 256)

In [71]:
model.predict_embeddings('CC').shape

(3, 256)

In [73]:
predicted = model.predict_from_sequences(valid_smiles[0:500])
count = 0
for s,p in zip(valid_smiles[0:500], predicted):
  if ''.join(p) == s:
    count += 1
print('reproduced', count, f'of {len(valid_smiles[0:500])} validation SMILES strings')

reproduced 0 of 500 validation SMILES strings


In [2]:
!ls ../datasets

CID-SMILES
VT-2005_Sigma_Profile_Database_Index_v2.xls
VT-2005_Sigma_Profiles_v2
VT_2005_sigma_smiles_cid.csv
melting_point.csv
melting_point_small.csv
patents_ochem_enamine_bradley_begstrom_training_.csv
pubchem_smiles.csv


In [12]:
import pandas as pd
file = "../datasets/CID-SMILES"
data = pd.read_csv(file, sep='\s+', header=None, names=["SMILES"])
data = pd.DataFrame(data, columns=["SMILES"])
data

,SMILES
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C
3,C1=CC(C(C(=C1)C(=O)O)O)O
4,CC(CN)O
5,C(C(=O)COP(=O)(O)O)N
...,...
162677696,C1[C@@H]([C@H](O[C@H]1N2C3=C(C4=NC=CN4C=N3)NC2...
162677697,CC(=O)O[C@H]1CO[C@@H]([C@H]([C@@H]1OC(=O)C)O)C...
162677698,C/C(=C\C=C\C(=C\C=C\C=C(/C)\C=C\C=C(\C=C\C=C(/...
162677699,C=[S@@](CC[C@@H](C(=O)O)N)C[C@@H]1[C@H]([C@H](...


In [13]:
data.to_csv("../datasets/pubchem_smiles.csv")

In [ ]:
loader = dc.data.CSVLoader()

In [4]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer="GraphConv")
train_dataset, valid_dataset, test_dataset = datasets

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [4]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

AttributeError: module 'deepchem.models' has no attribute 'GraphConvModel'

In [ ]:
model.fit(train_dataset,nb_epoch=5)
# for train in train_dataset.iterbatches(batch_size=100, epochs=10, deterministic=False):
#     model.fit(train)

In [1]:
# clean up original csv and save 
import pandas as pd

file ='patents_ochem_enamine_bradley_begstrom_training_.csv'
data = pd.read_csv(file,sep=',', header=0,error_bad_lines=False, names=["SMILES","CASRN","EXTERNALID","N","NAME1","NAME2","ARTICLEID","PUBMEDID","PAGE","TABLE","Melting Point","UNIT1 {Melting Point}","Melting Point {measured, converted}","UNIT2 {Melting Point}","Dataset","comment (property)","comment (chemical)","measurement method","Pressure","UNIT {Pressure}","Resp set numeric condition","UNIT {Resp set numeric condition}","Som set numeric condition","UNIT {Som set numeric condition}","Quality code","UNIT {Quality code}"])
data = pd.DataFrame(data)
# data
melting_point = data[["SMILES", 'Melting Point {measured, converted}']]
melting_point = melting_point.iloc[0:50000, :]
melting_point.rename(columns={"Melting Point {measured, converted}": "Melting Point"}, inplace=True)
melting_point.to_csv("melting point small.csv")

b'Skipping line 879: expected 26 fields, saw 28\nSkipping line 1317: expected 26 fields, saw 27\nSkipping line 1318: expected 26 fields, saw 27\nSkipping line 1319: expected 26 fields, saw 27\nSkipping line 2017: expected 26 fields, saw 28\nSkipping line 3688: expected 26 fields, saw 27\nSkipping line 3836: expected 26 fields, saw 27\nSkipping line 3838: expected 26 fields, saw 27\nSkipping line 9210: expected 26 fields, saw 27\nSkipping line 11121: expected 26 fields, saw 27\nSkipping line 11242: expected 26 fields, saw 28\nSkipping line 12638: expected 26 fields, saw 27\nSkipping line 12639: expected 26 fields, saw 27\nSkipping line 14751: expected 26 fields, saw 29\nSkipping line 15499: expected 26 fields, saw 28\nSkipping line 15673: expected 26 fields, saw 28\nSkipping line 15674: expected 26 fields, saw 28\nSkipping line 17568: expected 26 fields, saw 27\nSkipping line 17746: expected 26 fields, saw 27\nSkipping line 18981: expected 26 fields, saw 27\nSkipping line 23404: expecte

In [ ]:
import pandas as pd

file ='patents_ochem_enamine_bradley_begstrom_training_.csv'
data = pd.read_csv(file, header=0, error_bad_lines=False)
data = pd.DataFrame(data)
# data[['Melting Point {measured, converted}', "UNIT {Melting Point}"]]
len(data["Melting Point {measured, converted}"].notna())
# mp = data['Melting Point'].to_numpy()
# mp

In [15]:
lists = ["SMILES","CASRN","EXTERNALID","N","NAME1","NAME2","ARTICLEID","PUBMEDID","PAGE","TABLE","Melting Point","UNIT1 {Melting Point}","Melting Point {measured, converted}","UNIT2 {Melting Point}","Dataset","comment (property)","comment (chemical)","measurement method","Pressure","UNIT {Pressure}","Resp set numeric condition","UNIT {Resp set numeric condition}","Som set numeric condition","UNIT {Som set numeric condition}","Quality code","UNIT {Quality code}"]
len(lists)

26

In [4]:
import pandas as pd

csv_file ='melting point small.csv'
loader = dc.data.CSVLoader(["Melting Point"], feature_field="SMILES", featurizer=dc.feat.ConvMolFeaturizer())

dataset = loader.create_dataset(csv_file)
len(dataset)

50000

In [5]:
print(dataset)

<DiskDataset X.shape: (50000,), y.shape: (50000, 1), w.shape: (50000, 1), task_names: ['Melting Point']>


In [6]:
splitter = dc.splits.RandomSplitter()
train_set, valid_set, test_set = splitter.train_valid_test_split(dataset)

In [7]:
model_mp = dc.models.GraphConvModel(n_tasks=1, batch_size=50, mode='regression', dropout=0.2)

2022-01-26 09:59:16.441666: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-26 09:59:16.443527: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-26 09:59:16.467209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-01-26 09:59:16.468186: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-01-26 09:59:16.468228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-26 09:5

In [ ]:
model_mp.fit(train_set, nb_epoch=1)

In [ ]:
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CCC'])[0]
for feature, descriptor in zip(features[:10], rdkit_featurizer.descriptors):
    print(descriptor, feature)